# AI Agents Basics with AutoGen

Introduces core AutoGen concepts for orchestrating conversations between autonomous agents.

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
llm_config = {"model": "gpt-4o-mini"}

## Define single Agent

In [6]:
from autogen import ConversableAgent

In [12]:
agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

In [13]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke", "role": "user"}]
)
print(reply)

Why don't scientists trust atoms? 

Because they make up everything!


## Conversation between two Conversable Agents

In [14]:
cathy = ConversableAgent(
    name="cathy",
    system_message="Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

joe = ConversableAgent(
    name="joe",
    system_message=
      "Your name is Cathy and you are a stand-up comedian."
      "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

In [16]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling!",
    max_turns=2,
    #is_termination_msg=lambda msg: "I gotta go" in msg["content"]
    summary_method = "reflection_with_llm",
    summary_prompt="Summarize the conversation"
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling!

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Alright, let’s roll! You know, they say laughter is the best medicine. But if that's true, I must not have health insurance because my jokes are the only thing keeping me from going to the doctor! How about you? What’s your go-to joke?

--------------------------------------------------------------------------------
joe (to cathy):

Well, they say laughter is indeed the best medicine, but if that’s the case, I must be a pharmacist because I’m handing out these jokes like candy! You know what I say, if you're ever feeling down, just remember: every time you laugh, a diet soda gets its wings!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, Joe! That’s fantastic! You're handing out jokes like a pharmacist, huh? I love it! You must be the only one making people laug

In [18]:
import pprint
pprint.pprint(chat_result.chat_history)
pprint.pprint(chat_result.cost)
pprint.pprint(chat_result.summary) #summary based on above settings

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling!",
  'name': 'joe',
  'role': 'assistant'},
 {'content': 'Hey Joe! Alright, let’s roll! You know, they say laughter is the '
             "best medicine. But if that's true, I must not have health "
             'insurance because my jokes are the only thing keeping me from '
             'going to the doctor! How about you? What’s your go-to joke?',
  'name': 'cathy',
  'role': 'user'},
 {'content': 'Well, they say laughter is indeed the best medicine, but if '
             'that’s the case, I must be a pharmacist because I’m handing out '
             "these jokes like candy! You know what I say, if you're ever "
             'feeling down, just remember: every time you laugh, a diet soda '
             'gets its wings!',
  'name': 'joe',
  'role': 'assistant'},
 {'content': "Haha, Joe! That’s fantastic! You're handing out jokes like a "
             'pharmacist, huh? I love it! You must be the only one making '
             

## Using of tools

In [20]:
from typing import Annotated, Literal
Operator = Literal["+", "-", "*", "/"]

def calculator(a:int, b:int, operator: Annotated[Operator, "operator"]) -> int:
  """A simple calculator tool"""
  if operator == "+":
      return a + b
  elif operator == "-":
      return a - b
  else:
    raise ValueError("Invalid operator")

In [21]:
math_expert_agent = ConversableAgent(
    name="MathExpert",
    system_message=
      "You are very good at math "
      "and you help users to solve thei math problems.",
    llm_config=llm_config,
    function_map={"calculator": calculator},
    human_input_mode="NEVER"
)

In [22]:
math_expert_agent.register_for_llm(name="calculator", description="A tool to make math operations")(calculator)

In [23]:
import json
reply = math_expert_agent.generate_reply(
    messages=[{"content":"What is 2+2?", "role" : "user"}]
)

## Build a team of Agents (sketch)

**UserProxy**: to allow the user to comment on the report and ask the writer to refine it

**Planner**: to determine relevant info needed to complete the task

**Engineer**: to write code using the defined plan by the planner

**Executor**: to execute the code written by the engineer

**Writer**: to write report

In [25]:
import autogen

In [26]:
user_proxy = autogen.ConversableAgent(
    name="user_proxy",
    system_message="Give the task and send instrctions to writer to refine the blog post",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS"
)

...

In [27]:
grouptchat = autogen.GroupChat(
    agents=[user_proxy],
    messages=[],
    max_round=10
)

In [28]:
#to determinate which agent should be involved
manager = autogen.GroupChatManager(
    groupchat=grouptchat,
    llm_config=llm_config,
)

In [ ]:
task = "<Define a task>"

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task
)